In [1]:
!pip install wandb -q
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")

wandb.login(key = secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanish-jain140301 (tanish1403) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
!python --version

Python 3.11.13


In [3]:
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 -q
# !pip install mmcv-full==1.7.1 -f https://download.openmmlab.com/mmcv/dist/cu116/torch1.13.0/index.html -q
!pip install mmdet==2.28.2 -q
!pip install -U openmim -q
!mim install "mmengine>=0.7.0" -q
!pip install xmltodict -q  # For dataset conversion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 512.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.5.2 requires torch>=2.1.0, but you have torch 2.0.1+cu118 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.6 MB/s eta 0:00:00
     

In [4]:
!mim install 'mmcv==2.0.1'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 19.4 MB/s eta 0:00:00


In [5]:
!mim install 'mmcv-full'

Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 MB 21.4 MB/s eta 0:00:00


In [6]:
# Step 6: Prepare Dataset (SCCOS to DOTA format)
import os
import shutil
import random
import xmltodict
import tqdm
import math

dataset_path = "/kaggle/input/sccos-dataset/"
working_dir = "/kaggle/working/sccos_dota"
train_images_dir = os.path.join(working_dir, "train/images")
train_labels_dir = os.path.join(working_dir, "train/labels")
val_images_dir = os.path.join(working_dir, "val/images")
val_labels_dir = os.path.join(working_dir, "val/labels")
test_images_dir = os.path.join(working_dir, "test/images")
test_labels_dir = os.path.join(working_dir, "test/labels")

In [7]:
# Step 4: Clone and Install MMRotate 0.3.4
!git clone https://github.com/zhangpeng2001/nirnet.git /kaggle/working/mmrotate
%cd /kaggle/working/mmrotate
# !git checkout v0.3.4  # Ensure exact version
!pip install -r requirements/build.txt -q
!pip install -v -e . -q

Cloning into '/kaggle/working/mmrotate'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (333/333), done.
remote: Total 482 (delta 143), reused 474 (delta 135), pack-reused 0 (from 0)
Receiving objects: 100% (482/482), 11.52 MiB | 33.61 MiB/s, done.
Resolving deltas: 100% (143/143), done.
/kaggle/working/mmrotate
Obtaining file:///kaggle/working/mmrotate
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 7.7 MB/s eta 0:00:00
  Running setup.py develop for mmrotate


In [8]:
# Step 5: Verify Installations
!pip list | grep -E 'torch|mmcv|mmdet|mmengine|mmrotate'

mmcv                                  2.0.1
mmcv-full                             1.7.2
mmdet                                 2.28.2
mmengine                              0.10.7
mmrotate                              0.3.4               /kaggle/working/mmrotate
pytorch-ignite                        0.5.2
pytorch-lightning                     2.5.2
torch                                 2.0.1+cu118
torchao                               0.10.0
torchaudio                            2.0.2+cu118
torchdata                             0.11.0
torchinfo                             1.8.0
torchmetrics                          1.7.3
torchsummary                          1.5.1
torchtune                             0.6.1
torchvision                           0.15.2+cu118


In [9]:
%%writefile mmrotate/models/necks/nirnet.py
import torch

from mmcv.cnn import ConvModule
from mmcv.runner import auto_fp16
from mmdet.models.necks import FPN
from mmcv.cnn.bricks.transformer import MultiheadAttention

from ..builder import ROTATED_NECKS


@ROTATED_NECKS.register_module()
class NIRNet(FPN):

    def __init__(self,
                 in_channels,
                 out_channels,
                 num_outs,
                 num_groups=4,  # New: For group attention
                 conv_cfg=None,
                 norm_cfg=None,
                 act_cfg=None,
                 **kwargs):
        super(NIRNet, self).__init__(
            in_channels,
            out_channels,
            num_outs,
            conv_cfg=conv_cfg,
            norm_cfg=norm_cfg,
            act_cfg=act_cfg,
            **kwargs)

        self.num_groups = num_groups  # New: Group attention param

        self.encoder_conv = ConvModule(
            out_channels,
            out_channels,
            3,
            padding=1,
            conv_cfg=conv_cfg,
            norm_cfg=norm_cfg,
            act_cfg=act_cfg,
            groups=out_channels,
            inplace=False)
        self.fusion_conv = ConvModule(
            out_channels,
            out_channels,
            3,
            padding=1,
            conv_cfg=conv_cfg,
            norm_cfg=norm_cfg,
            act_cfg=act_cfg,
            inplace=False)
        self.excite_conv = ConvModule(
            out_channels,
            out_channels,
            3,
            padding=1,
            conv_cfg=conv_cfg,
            norm_cfg=norm_cfg,
            act_cfg=act_cfg,
            inplace=False)
        self.sigmoid = torch.nn.Sigmoid()
        self.fcm_ip_channel_fc = torch.nn.Linear(out_channels // 2, 1)
        self.fcm_ip_channel_atten = MultiheadAttention(embed_dims=16, num_heads=8)
        self.fcm_sp_channel_atten = MultiheadAttention(embed_dims=16, num_heads=8)
        self.fcm_sp_channel_fc = torch.nn.Linear(out_channels // 2, 1)

        # New: Lightweight group attention convs (one per group, shared across paths for efficiency)
        self.group_attn_convs = torch.nn.ModuleList([
            ConvModule(
                out_channels // (2 * num_groups),  # Split channels further for IP/SP
                out_channels // (2 * num_groups),
                1,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=None) for _ in range(num_groups)
        ])

    @auto_fp16()
    def forward(self, inputs):
        outs = super(NIRNet, self).forward(inputs)
        outs = self.dpic(outs)
        return tuple(outs)

    def dpic(self, feats):

        encoder_feats = []
        
        for feat in feats:
            split_feat = self.encoder_conv(feat)
            dw_feat, pw_feat = torch.split(split_feat, split_size_or_sections=128, dim=1)
            # Enhanced NPM with frequency
            weight1, weight2 = self.fp_npm(feat)
            # Enhanced FCM with group attention
            fcm_sp_feat = self.fcm_sp(pw_feat * (1 + weight1))
            dw_feat = dw_feat * (1 + weight2)
            pw_dw_features = torch.cat([fcm_sp_feat, dw_feat])
            fcm_ip_feat = self.fcm_ip(dw_feat)
            # Concatenation and fusion
            fusion_feat = self.fusion_conv(torch.cat([fcm_ip_feat , fcm_sp_feat], dim=1))
            encoder_feats.append(fusion_feat + feat)
            
        return encoder_feats
    
    def fcm_ip(self, feat):
        # Optimal mask
        pixel_feat = torch.max(feat, 1, keepdim=True)[0]
        pixel_feat = pixel_feat + self.fcm_ip_channel_fc(feat.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        # Optimal descriptor
        flatten_feat = feat.view(feat.size(0), feat.size(1), -1)
        channel_feat_mean = torch.mean(flatten_feat, 2, keepdim=True).view(feat.size(0), 8, 16)
        channel_feat_max = torch.max(flatten_feat, 2, keepdim=True)[0].view(feat.size(0), 8, 16)
        channel_feat = self.fcm_ip_channel_atten(channel_feat_mean, channel_feat_mean, channel_feat_max)
        channel_feat = channel_feat.view(channel_feat.size(0), -1).unsqueeze(-1).unsqueeze(-1)
        # New: Group attention
        channel_feat = self._apply_group_attention(channel_feat)
        # Optimal feature
        fcm_ip_feat = pixel_feat * channel_feat 
        return fcm_ip_feat
    
    def fcm_sp(self, feat):
        # Holistic mask
        pixel_feat = torch.mean(feat, 1, keepdim=True)
        pixel_feat = pixel_feat + self.fcm_sp_channel_fc(feat.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        # Holistic descriptor
        flatten_feat = feat.view(feat.size(0), feat.size(1), -1)
        channel_feat_mean = self.sigmoid(torch.mean(flatten_feat, 2, keepdim=True).view(feat.size(0), 8, 16))
        channel_feat_max = self.sigmoid(torch.max(flatten_feat, 2, keepdim=True)[0].view(feat.size(0), 8, 16))
        channel_feat = self.fcm_sp_channel_atten(channel_feat_mean, channel_feat_mean, channel_feat_max)
        channel_feat = channel_feat.view(channel_feat.size(0), -1).unsqueeze(-1).unsqueeze(-1)
        # New: Group attention
        channel_feat = self._apply_group_attention(channel_feat)
        # Holistic feature
        fcm_sp_feat = pixel_feat * channel_feat
        return fcm_sp_feat

    def fp_npm(self, feat):
        # Original spatial min
        pixel_feat = torch.min(feat, 1, keepdim=True)[0]
        channel_feat = torch.min(torch.min(feat, 2, keepdim=True)[0], 3, keepdim=True)[0]
        pcmin_feat = pixel_feat * channel_feat
        # New: Frequency perception (low-freq blur, high-freq residual)
        low_freq = torch.nn.functional.avg_pool2d(feat, kernel_size=3, stride=1, padding=1)
        high_freq = feat - low_freq
        freq_min = torch.min(low_freq, high_freq)
        # Fuse spatial + freq
        pcmin_feat = pcmin_feat + freq_min  # Simple addition for fusion
        excitation = self.excite_conv(pcmin_feat)
        excitation = self.sigmoid(excitation)
        excitation1, excitation2 = torch.split(excitation, split_size_or_sections=128, dim=1)
        return excitation1, excitation2

    # New: Helper for group attention
    def _apply_group_attention(self, feat):
        groups = torch.chunk(feat, self.num_groups, dim=1)  # Split channels
        attn_groups = [self.sigmoid(self.group_attn_convs[i](g)) * g for i, g in enumerate(groups)]
        return torch.cat(attn_groups, dim=1)  # Fuse back

Overwriting mmrotate/models/necks/nirnet.py


In [10]:
import os
os.makedirs("/kaggle/working/mmrotate/configs/firnet", exist_ok = True)

In [11]:
%%writefile configs/firnet/firnet_r50_fpn_1x_sccos.py
_base_ = [
    '../rotated_retinanet/rotated_retinanet_obb_r50_fpn_1x_dota_le135.py'
]

angle_version = 'le135'
model = dict(
    neck=dict(
        type='NIRNet',  # Changed to FIRNet
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    bbox_head=dict(
        reg_decoded_bbox=True,
        num_classes=1,
        loss_bbox=dict(type='GDLoss', loss_type='gwd', loss_weight=5.0)))

dataset_type = 'DOTADataset'
# data_root = '/kaggle/working/sccos_dota/'
data_root = '/kaggle/input/sccos-dota/sccos_dota/'
classes = ('ship',)  # Explicitly define classes
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RResize', img_scale=(1024, 1024)),
    dict(
        type='RRandomFlip',
        flip_ratio=[0.25, 0.25, 0.25],
        direction=['horizontal', 'vertical', 'diagonal'],
        version='le135'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1024, 1024),
        flip=False,
        transforms=[
            dict(type='RResize'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='DefaultFormatBundle'),
            dict(type='Collect', keys=['img'])
        ])
]
data = dict(
    samples_per_gpu=4,
    workers_per_gpu=2,
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train/labels/',
        img_prefix=data_root + 'train/images/',
        pipeline=train_pipeline,
        version='le135',
        classes=('ship',)),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'val/labels/',
        img_prefix=data_root + 'val/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test/labels/',
        img_prefix=data_root + 'test/images/',
        pipeline=test_pipeline,
        version='le135',
        classes=('ship',)))
evaluation = dict(interval=1, metric='mAP', save_best='mAP') # Add save_best here
lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=1000,
    warmup_ratio=0.3333333333333333,
    step=[8, 11])
runner = dict(type='EpochBasedRunner', max_epochs=25)
checkpoint_config = dict(interval=1) # Keep interval for saving
log_config = dict(
    interval=100,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='WandbLoggerHook',
             init_kwargs=dict(
                 project='FREANet_training',
                 name='FREANet_rr'),
             interval=100)
    ])

Writing configs/firnet/firnet_r50_fpn_1x_sccos.py


In [12]:
%cd /kaggle/working/mmrotate
!python tools/analysis_tools/get_flops.py configs/firnet/firnet_r50_fpn_1x_sccos.py

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
RotatedRetinaNet(
  37.312 M, 99.994% Params, 235.395 GFLOPs, 100.000% FLOPs, 
  (backbone): ResNet(
    23.283 M, 62.396% Params, 86.095 GFLOPs, 36.575% FLOPs, 
    (conv1): Conv2d(0.0 M, 0.000% Params, 2.466 GFLOPs, 1.048% FLOPs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(0.0 M, 0.000% Params, 0.034 GFLOPs, 0.014% FLOPs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(0.0 M, 0.000% Params, 0.017 GFLOPs, 0.007% FLOPs, inplace=True)
    (maxpool): MaxPool2d(0.0

In [13]:
# Step 8: Train the Model
%cd /kaggle/working/mmrotate
!mkdir -p /kaggle/working/runs/FIRNet_train


!python tools/train.py \
    configs/firnet/firnet_r50_fpn_1x_sccos.py \
    --work-dir /kaggle/working/runs/FIRNet_train \
    --gpus 1 \


print("Training completed.")

/kaggle/working/mmrotate
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your 

In [14]:
# Test FIRNet (uncomment to run)
import os
import glob

%cd /kaggle/working/mmrotate

# Step 1: Find the best checkpoint file automatically
checkpoint_dir = '/kaggle/working/runs/FIRNet_train'
best_checkpoint_path = None
# Search for the best checkpoint file
for filename in glob.glob(os.path.join(checkpoint_dir, 'best_mAP_*.pth')):
    best_checkpoint_path = filename
    break  # Take the first one found

if best_checkpoint_path:
    print(f"Found best checkpoint: {best_checkpoint_path}")
    
    # Step 2: Run the test command with the found checkpoint
    !mkdir -p /kaggle/working/runs/FIRNet_test
    !python tools/test.py \
        configs/firnet/firnet_r50_fpn_1x_sccos.py \
        {best_checkpoint_path} \
        --eval mAP \
        --out /kaggle/working/runs/firnet_test/results.pkl \
        --show-dir /kaggle/working/runs/firnet_test/vis
    
    print("FIRNet testing completed. Results saved.")
else:
    print(f"Error: No best checkpoint found in {checkpoint_dir}. Please ensure training was successful.")

/kaggle/working/mmrotate
Found best checkpoint: /kaggle/working/runs/FIRNet_train/best_mAP_epoch_23.pth
/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:38: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/kaggle/working/mmrotate/mmrotate/utils/setup_env.py:48: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system bein